In [5]:
path = "C:\\Users\\Happy\\Desktop\\빅분기\\input"
# .iloc로 하기 df[0] => 인덱스로 확인함..

# T2-1
생존여부 예측모델 만들기
학습용 데이터 (X_train, y_train)을 이용하여 생존 예측 모형을 만든 후, 이를 평가용 데이터(X_test)에 적용하여 얻은 예측값을 다음과 같은 형식의 CSV파일로 생성하시오(제출한 모델의 성능은 accuracy 평가지표에 따라 채점)
(가) 제공 데이터 목록

y_train: 생존여부(학습용)
X_trian, X_test : 승객 정보 (학습용 및 평가용)
(나) 데이터 형식 및 내용

y_trian (712명 데이터)
시험환경 세팅은 예시문제와 동일한 형태의 X_train, y_train, X_test 데이터를 만들기 위함임

유의사항
성능이 우수한 예측모형을 구축하기 위해서는 적절한 데이터 전처리, 피처엔지니어링, 분류알고리즘, 하이퍼파라미터 튜닝, 모형 앙상블 등이 수반되어야 한다.
수험번호.csv파일이 만들어지도록 코드를 제출한다.
제출한 모델의 성능은 accuracy로 평가함


In [6]:
# 시험환경 세팅 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2021)
    
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv(path+ "\\titanic\\train.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='Survived', id_name='PassengerId')

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((712, 11), (179, 11), (712, 2), (179, 2))

In [7]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np

x_len = len(X_train)
X_all = pd.concat([X_train,X_test] , axis=0)
# train_all = pd.concat([X_train,y_train] , axis=1)

# X_train = X_all[train_list][:x_len]
# X_test = X_all[train_list][x_len:]

X_all = pd.get_dummies(X_all)

X_train = X_all[:x_len]
X_test = X_all[x_len:]

train_all = pd.concat([X_train,y_train] , axis=1)

aa = pd.DataFrame(abs(train_all.corr()['Survived']))
train_list = list(aa.loc[(aa['Survived']>0.2)][:-1].index)

X_train = X_train[train_list]
X_test = X_test[train_list]

X_all = pd.concat([X_train,X_test] , axis=0)

from sklearn.preprocessing import StandardScaler
SS = StandardScaler()
X_all = SS.fit_transform(X_all)

X_train = X_all[:x_len]
X_test = X_all[x_len:]



In [4]:
from xgboost import XGBClassifier
model = XGBClassifier()
# model = XGBClassifier(learning_rate=0.01,objective= 'binary:logistic',eval_metric = "merror", booster='gbtree',use_label_encoder= True,)
model.fit(X_train,y_train['Survived'],verbose = 1) #@ fit에 ,verbose = 1 로 두면 
print(model.score(X_train,y_train['Survived']))

pred = model.predict(X_test)
#@@ 저장 부분 잘 기억하기
pd.DataFrame({'cust_id': y_test.PassengerId, 'Survived': pred}).to_csv('수험번호.csv', index=False)
# pd.read_csv('수험번호.csv')

print(model.score(X_test,y_test['Survived']))

0.901685393258427
0.776536312849162


In [8]:
# from lightgbm import LGBMClassifier
# model = LGBMClassifier(learning_rate = 0.001)
# # model = XGBClassifier(learning_rate=0.01,objective= 'binary:logistic',eval_metric = "merror", booster='gbtree',use_label_encoder= True,)
# model.fit(X_train,y_train['Survived'],verbose = 1) #@ fit에 ,verbose = 1 로 두면 
# print(model.score(X_train,y_train['Survived']))

# pred = model.predict(X_test)
# #@@ 저장 부분 잘 기억하기
# pd.DataFrame({'cust_id': y_test.PassengerId, 'Survived': pred}).to_csv('수험번호.csv', index=False)
# # pd.read_csv('수험번호.csv')

# print(model.score(X_test,y_test['Survived']))

0.6193820224719101
0.6033519553072626


C:\Users\Happy\anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


In [5]:
print(model.score(X_test,y_test['Survived']))
import sklearn.metrics

print(sklearn.metrics.f1_score(y_test['Survived'],pred), \
sklearn.metrics.recall_score(y_test['Survived'],pred), \
sklearn.metrics.r2_score(y_test['Survived'],pred), \
sklearn.metrics.multilabel_confusion_matrix(y_test['Survived'],pred))

from sklearn.metrics import roc_auc_score
sklearn.metrics.roc_auc_score(y_test['Survived'],pred)


0.776536312849162
0.7183098591549296 0.7183098591549296 0.06624934793948867 [[[51 20]
  [20 88]]

 [[88 20]
  [20 51]]]


0.7665623369848722

In [6]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X_train,y_train['Survived'])

print(model.score(X_train,y_train['Survived']))

pred = model.predict(X_test)

print(model.score(X_test,y_test['Survived']))

0.9199438202247191
0.7541899441340782


In [7]:
1.더미화
2.상관분석 후 필요한 컬럼만 뽑음
3.필요한 컬럼들 정규화.
4.예측

SyntaxError: invalid syntax (<ipython-input-7-16a5598eed0d>, line 1)

In [ ]:
## 예전 내 정답
import pandas as pd
df = pd.read_csv(path+ "\\titanic\\train.csv")

from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(df, test_size=0.2, shuffle=True, random_state=2021)
y_train = X_train[['PassengerId', 'Survived']]
X_train = X_train.drop(columns=['PassengerId', 'Survived'])
y_test = X_test[['PassengerId', 'Survived']]
X_test = X_test.drop(columns=['PassengerId', 'Survived'])


X_train_len = len(X_train)
# print(y_train)

X_all = pd.concat([X_train,X_test],axis= 0)
# print(y_test.head())
X_all_1 = pd.get_dummies(X_all)
###시간남으면 하기####

X_train_1 = X_all_1[:X_train_len]
X_test_1 = X_all_1[X_train_len:]
train_all = pd.concat([X_train_1,y_train['Survived']],axis = 1)
train_all_1 = pd.DataFrame(train_all.corr()['Survived'])
# print(train_all_1[abs(train_all_1['Survived'])>0.01][:-1].index)
Useing_col = list(train_all_1[abs(train_all_1['Survived'])>0.02][:-1].index)
# print(len(Useing_col))
X_train_1 = X_train_1[Useing_col]
X_test_1 = X_test_1[Useing_col]
X_all_1 = pd.concat([X_train_1,X_test_1],axis = 0)
# print(X_all_1.info())
# ###시간남으면 하기####

from sklearn.preprocessing import StandardScaler
SS = StandardScaler()
X_all_2 = SS.fit_transform(X_all_1)

X_train_2 = X_all_2[:X_train_len]
X_test_2 = X_all_2[X_train_len:]
# print(X_all_1.shape)
# print(y_train['Survived'].shape)

from xgboost import XGBClassifier
model = XGBClassifier(eval_metric =  "logloss",use_label_encoder = False)
model.fit(X_train_2,y_train['Survived'])
print(model.score(X_train_2,y_train['Survived']))

pred = model.predict(X_test_2)

pd.DataFrame({'PassengerId':y_test.PassengerId,'pred or prob':pred}).to_csv("수험번호.csv",index = False)

from sklearn.metrics import roc_auc_score
print(roc_auc_score(model.predict(X_train_2),y_train['Survived']))
# print(dir(sklearn.metrics))

##여기는 수험자가 확인 불가능 영역
print(model.score(X_test_2,y_test['Survived']))
print(roc_auc_score(pred,y_test['Survived']))

In [ ]:
y_test['Survived']

# T2-2. Pima Indians Diabetes(피마 인디언 당뇨병)

In [9]:
# 시험환경 세팅 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2021)
    
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv(path+ "\\archive\\diabetes.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='Outcome')

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((614, 9), (154, 9), (614, 2), (154, 2))

In [24]:
tr_len = len(X_train) 

X_all = pd.concat([X_train, X_test],axis = 0)
tr_col = X_all.columns

X_all_1 = pd.get_dummies(X_all)

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
sclaer = MinMaxScaler()
X_all_2 = sclaer.fit_transform(X_all_1)
X_all_2 = pd.DataFrame(X_all_2, columns =tr_col )

del X_all_2['id'] #id는 의미가 있을리가 없다

X_train_2 = X_all_2[:tr_len]
X_test_2 = X_all_2[tr_len:]

y_train.reset_index(inplace = True,drop=True) # @@ 인덱스 번호로 concat(합쳐짐)합쳐지기 때문에 리셋 인덱스를 해야함
train_all = pd.concat([X_train_2,y_train[['Outcome']]],axis = 1)

print(train_all.corr()['Outcome']) # 상관성이 다들 어느정도 있어 보이니 넘어가자.


from xgboost import XGBClassifier
model = XGBClassifier(learning_rate = 0.01)
model.fit(X_train_2 ,y_train['Outcome'], eval_metric  = 'logloss' )

# model = XGBClassifier()
# model.fit(X_train_2 ,y_train['Outcome'])

print(model.score(X_train_2 ,y_train['Outcome']))

pred = model.predict(X_test_2)

answer = pd.DataFrame(pred,columns=["predict"])
answer.to_csv("수험번호.csv")

Pregnancies                 0.232571
Glucose                     0.458378
BloodPressure               0.048623
SkinThickness               0.075839
Insulin                     0.128257
BMI                         0.296715
DiabetesPedigreeFunction    0.174323
Age                         0.218914
Outcome                     1.000000
Name: Outcome, dtype: float64
0.9039087947882736


c:\users\happy\appdata\local\programs\python\python37\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [25]:
print(model.score(X_test_2,y_test['Outcome']))
import sklearn.metrics

print(sklearn.metrics.f1_score(y_test['Outcome'],pred), \
sklearn.metrics.recall_score(y_test['Outcome'],pred), \
sklearn.metrics.r2_score(y_test['Outcome'],pred), \
sklearn.metrics.multilabel_confusion_matrix(y_test['Outcome'],pred))

from sklearn.metrics import roc_auc_score
sklearn.metrics.roc_auc_score(y_test['Outcome'],pred)


0.7532467532467533
0.6346153846153846 0.5789473684210527 -0.05841924398625409 [[[33 24]
  [14 83]]

 [[83 14]
  [24 33]]]


0.7173087357569181

# T2-3. Adult Census Income Tutorial
age: 나이
workclass: 고용 형태
fnlwgt: 사람의 대표성을 나타내는 가중치(final weight)
education: 교육 수준
education.num: 교육 수준 수치
marital.status: 결혼 상태
occupation: 업종
relationship: 가족 관계
race: 인종
sex: 성별
capital.gain: 양도 소득
capital.loss: 양도 손실
hours.per.week: 주당 근무 시간
native.country: 국적
income: 수익 (예측해야 하는 값)

In [204]:
# 시험환경 세팅 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2021)
    
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv(path+ "\\archive\\adult.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='income', null_name='?')

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((26048, 15), (6513, 15), (26048, 2), (6513, 2))

In [205]:
import pandas as pd
import numpy as np

In [220]:

X_tr_len = len(X_train) # 인덱스
X_all = pd.concat([X_train,X_test])
del X_all['id'] # 무조건 필요없는 id컬럼 드롭


X_all = X_all.reset_index(drop=True)

X_all_1 = pd.get_dummies(X_all)

X_tr_cal = X_all_1.columns # 컬럼

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

Scaler = StandardScaler()
X_all_2 = Scaler.fit_transform(X_all_1)
X_all_2 = pd.DataFrame(data =X_all_2, columns = X_tr_cal)

X_train_2 = X_all_2[:X_tr_len]
X_test_2 = X_all_2[X_tr_len:]

y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

# y_train.loc[y_train['income']=="<=50K",'income'] = int(0)
# y_train.loc[y_train['income']==">50K",'income'] = int(1)

# y_test.loc[y_test['income']=="<=50K",'income'] = int(0)
# y_test.loc[y_test['income']==">50K",'income'] = int(1)

from xgboost import XGBClassifier
model = XGBClassifier()
# model = XGBClassifier(learning_rate=0.01,objective= 'binary:logistic',eval_metric = "merror", booster='gbtree',use_label_encoder= True,)
model.fit(X_train_2,y_train['income'])
print(model.score(X_train_2,y_train['income']))
print(model.score(X_test_2,y_test['income']))
print(model)
pred = model.predict(X_test_2)
print(pred)
pd.DataFrame({"pred":pred}).to_csv("수험번호.csv",index= False)

c:\users\happy\appdata\local\programs\python\python37\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:33:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.9058277027027027
0.871641332719177
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
['>50K' '<=50K'

In [209]:
y_test.loc[y_test['income'] == "<=50K",'out_put'] = 0
y_test.loc[y_test['income'] != "<=50K",'out_put'] = 1

pred = pd.read_csv("수험번호.csv")

pred.loc[pred['pred'] == "<=50K",'out_put'] = 0
pred.loc[pred['pred'] != "<=50K",'out_put'] = 1

In [219]:
print("모델점수",model.score(X_test_2,y_test['income']))
aa , bb = y_test['out_put'],pred['out_put']
import sklearn.metrics

print("f1_score",sklearn.metrics.f1_score(aa , bb), \
"재현율",sklearn.metrics.recall_score(aa , bb), \
"r2_score",sklearn.metrics.r2_score(aa , bb), \
"혼동행렬",sklearn.metrics.multilabel_confusion_matrix(aa , bb))

from sklearn.metrics import roc_auc_score
sklearn.metrics.roc_auc_score(aa , bb)



f1_score 0.7078965758211041 재현율 0.6539703034215623 r2_score 0.29188439527672205 혼동행렬 [[[1013  536]
  [ 300 4664]]

 [[4664  300]
  [ 536 1013]]]


0.7967675852321349

# T2-4. House Prices (Regression)


In [9]:
# 시험환경 세팅 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, shuffle=True, random_state=2021)
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[id_name, target])
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[id_name, target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv(path+ "\\house-prices-advanced-regression-techniques\\train.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='SalePrice', id_name='Id')


In [10]:
X_train_len = len(X_train)
X_all = pd.concat( [X_train,X_test])

X_all_1 = pd.get_dummies(X_all)


In [11]:
from sklearn.preprocessing import StandardScaler
Scaler = StandardScaler()
X_all_col = X_all_1.columns
X_all_2 = Scaler.fit_transform(X_all_1)
X_all_2 = pd.DataFrame(X_all_2, columns = X_all_col)

X_train_2 = X_all_2[:X_train_len]
X_test_2 = X_all_2[X_train_len:]

y_train_1 = y_train.reset_index(drop= True)
y_test_1 = y_test.reset_index(drop= True)

Train_all = pd.concat([X_train_2,y_train_1],axis = 1)
aa = pd.DataFrame(abs(Train_all.corr()['SalePrice'])>0.1)
use_columns = aa.loc[aa['SalePrice']==True][:-1].index

X_train_3 = X_train_2[use_columns]
X_test_3 = X_test_2[use_columns]

In [12]:
from sklearn.ensemble import AdaBoostRegressor
model = AdaBoostRegressor()
model.fit(X_train_3,y_train_1['SalePrice'])

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [13]:
from xgboost import XGBRegressor
model = XGBRegressor(learning_rate = 0.3,booster = "gblinear", reg_lambda = 1.2, reg_alpha = 0.2) # @@ gbliner를 이용하자
# model = XGBRegressor()
model.fit(X_train_3,y_train_1['SalePrice'])

print(model.score(X_train_3,y_train_1['SalePrice']))

pred = model.predict(X_test_3)

pd.DataFrame({"predict":pred}).to_csv("수험번호.csv", index = False)
# pred

print(model.score(X_test_3,y_test_1['SalePrice']))

from sklearn.metrics import mean_squared_error
print(np.sqrt(mean_squared_error(y_train_1['SalePrice'], model.predict(X_train_3)))) # 정답
print(np.sqrt(mean_squared_error(y_test_1['SalePrice'], pred))) # 정답

0.8491270042093133
0.8502719975650403
31124.538902331296
29559.060750897082


In [14]:
from xgboost import XGBRegressor
model = XGBRegressor(learning_rate = 0.1,booster = "gblinear") # @@ gbliner를 이용하자
model = XGBRegressor()
model.fit(X_train_2,y_train_1['SalePrice'])

print(model.score(X_train_2,y_train_1['SalePrice']))

pred = model.predict(X_test_2)
# pred

print(model.score(X_test_2,y_test_1['SalePrice']))

from sklearn.metrics import mean_squared_error
print(np.sqrt(mean_squared_error(y_train_1['SalePrice'], model.predict(X_train_2)))) # 정답
print(np.sqrt(mean_squared_error(y_test_1['SalePrice'], pred))) # 정답

0.9997592786239734
0.9074264332872578
1243.2386352266383
23242.48788008825


# T2-5. Insurance_Starter (Tutorial)


In [157]:
# 시험환경 세팅 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2021)
    
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv(path+ "\\archive\\insurance.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='charges')

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1070, 7), (268, 7), (1070, 2), (268, 2))

In [163]:
X_train_len = len(X_train)

X_all = pd.concat([X_train,X_test],axis = 0)
X_all_1 = pd.get_dummies(X_all)

x_train = X_all_1[:X_train_len]
x_test = X_all_1[X_train_len:]

train_all = pd.concat([x_train,y_train],axis = 1)
df1 = pd.DataFrame(abs(train_all.corr()['charges'])>0.007)
train_col = df1.loc[df1['charges']==True].index[:-1]
X_all_2 = X_all_1[train_col]

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

Scaler = MinMaxScaler()
X_all_3 = Scaler.fit_transform(X_all_2)
X_all_3 = pd.DataFrame(X_all_3,columns = train_col)

x_train = X_all_3[:X_train_len]
x_test = X_all_3[X_train_len:]
# y_train.shape, y_test.shape

# y_train = y_train.reset_index(drop=True)
# y_test = y_test.reset_index(drop=True)

from xgboost import XGBRegressor

model = XGBRegressor()
model.fit(x_train,y_train['charges'])
print(model.score(x_train,y_train['charges']))
pred = model.predict(x_test)
pd.DataFrame({"predict":pred}).to_csv("수험번호.csv",index= False)

print(model.score(x_test,y_test['charges']))

0.9964181737350261
0.8120888367117656


In [155]:
from xgboost import XGBRegressor
# model = XGBRegressor(learning_rate = 0.1,booster = "gblinear") # @@ gbliner를 이용하자??
model = XGBRegressor()
# model = XGBRegressor(booster = "gblinear")
model.fit(x_train,y_train['charges'])

print(model.score(x_train,y_train['charges']))

pred = model.predict(x_test)
# pred

print(model.score(x_test,y_test['charges']))

from sklearn.metrics import mean_squared_error
print(np.sqrt(mean_squared_error(y_train['charges'], model.predict(x_train)))) # 정답
print(np.sqrt(mean_squared_error(y_test['charges'], pred))) # 정답

0.9964181737350261
0.8120559186498852


# T2-6. Bike-sharing-demand (Regression)_2[joge]

In [214]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, shuffle=True, random_state=2021)
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[id_name, target])
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[id_name, target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv(path+ "\\bike\\train.csv")
    
X_train, X_test, y_train, y_test = exam_data_load(df, target='count')#, id_name='Id')

In [216]:
X_train_len = len(X_train)
x_all = pd.concat([X_train,X_test],axis = 0)

x_all['datetime'] = pd.to_datetime(x_all['datetime'])


x_all_1 = pd.get_dummies(x_all)
del x_all_1['datetime']
X_columns = x_all_1.columns

x_all_1 = pd.DataFrame(x_all_1 , columns = X_columns)

from sklearn.preprocessing import StandardScaler
Scaler = StandardScaler()
x_all_2 = Scaler.fit_transform(x_all_1)
# x_all_2 = pd.DataFrame(x_all_2 , columns = X_columns)

x_train =  x_all_2[:X_train_len]
x_test = x_all_2[X_train_len:]

In [236]:
from xgboost import XGBRegressor
model = XGBRegressor()
model.fit(x_train,y_train['count'])
pred = model.predict(x_test)
print(model.score(x_train,y_train['count']))

print(model.score(x_test,y_test['count']))

0.9999352385395511
0.9996688076004838


In [262]:
df = pd.DataFrame({"index":y_test.id,"pred":pred})
df = df.set_index(df['index'],drop = True,inplace=False)
del df["index"]
df.to_csv("수험번호.csv",index = True)

In [238]:
from xgboost import XGBRegressor
# model = XGBRegressor(learning_rate = 0.1,booster = "gblinear") # @@ gbliner를 이용하자??
model = XGBRegressor()
# model = XGBRegressor(booster = "gblinear")
model.fit(x_train,y_train['count'])

print(model.score(x_train,y_train['count']))

pred = model.predict(x_test)
# pred

print(model.score(x_test,y_test['count']))

from sklearn.metrics import mean_squared_error
print(np.sqrt(mean_squared_error(y_train['count'], model.predict(x_train)))) # 정답
print(np.sqrt(mean_squared_error(y_test['count'], pred))) # 정답

0.9999352385395511
0.9996688076004838
1.4631759227449768
3.246032707799647


In [ ]:
# 캐글 풀이
# # print(X_train.head())
# # print(y_train.head())

# X_train_len = len(X_train)
# X_all = pd.concat([X_train,X_test],axis = 0)

# # print(X_all.info())
# # print(X_all["datetime"])
# # print(X_all.head())
# # print(X_all.tail())
# # print(X_all.columns)

# # print(pd.to_datetime(X_all["datetime"]))
# Day_time = X_all[["datetime"]]
# X_all = X_all.drop(["datetime"],axis = 1)
# X_all_1 = pd.get_dummies(X_all)

# # X_all_1 = pd.concat([X_all_1,Day_time],axis = 1)
# print(X_all_1.head())

# from sklearn.preprocessing import StandardScaler
# SS = StandardScaler()
# X_all_2 = SS.fit_transform(X_all_1)



# X_train_2 = X_all_2[:X_train_len]
# X_test_2 = X_all_2[X_train_len:]

# from xgboost import XGBRegressor
# model = XGBRegressor()

# model.fit(X_train_2,y_train["count"])

# print(model.score(X_train_2,y_train["count"]))

# pred = model.predict(X_test_2)

# print(model.score(X_test_2,y_test["count"]))

# [MOCK EXAM1] TYPE2. HR-DATA / 작업형2 모의고사

In [80]:
import pandas as pd
X_test = pd.read_csv(path+ "\\hr_data\\X_test.csv")
X_train = pd.read_csv(path+ "\\hr_data\\X_train.csv")
y_train = pd.read_csv(path+ "\\hr_data\\y_train.csv")

In [138]:
tr_len = len(X_train)
x_all = pd.concat([X_train , X_test],axis = 0)

del x_all['enrollee_id']

x_all_1 = pd.get_dummies(x_all)
X_columns = x_all_1.columns

from sklearn.preprocessing import StandardScaler # @ 
Scaler = StandardScaler()
Scaler.fit(x_all_1)
x_all_2 = Scaler.transform(x_all_1)

# x_all_2 = pd.DataFrame(x_all_2 ,columns = X_columns)
x_train = x_all_2[:tr_len]
x_test = x_all_2[tr_len:]



c:\users\happy\appdata\local\programs\python\python37\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:34:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.8521244388767095


In [147]:
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(x_train , y_train["target"])
print(model.score(x_train , y_train["target"]))

pred = model.predict(x_test)
df = pd.DataFrame({"predict":pred})

df['index'] = y_train['enrollee_id']
df = df.set_index(df["index"],drop = True,inplace = False)
df.to_csv("수험번호.csv",index = False)

c:\users\happy\appdata\local\programs\python\python37\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:35:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.8521244388767095


In [154]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()

model.fit(x_train , y_train["target"])
print(model.score(x_train , y_train["target"]))

pred = model.predict(x_test)

0.9985906670842468


In [156]:
dir(sklearn.ensemble)

['AdaBoostClassifier',
 'AdaBoostRegressor',
 'BaggingClassifier',
 'BaggingRegressor',
 'BaseEnsemble',
 'ExtraTreesClassifier',
 'ExtraTreesRegressor',
 'GradientBoostingClassifier',
 'GradientBoostingRegressor',
 'HistGradientBoostingClassifier',
 'HistGradientBoostingRegressor',
 'IsolationForest',
 'RandomForestClassifier',
 'RandomForestRegressor',
 'RandomTreesEmbedding',
 'StackingClassifier',
 'StackingRegressor',
 'VotingClassifier',
 'VotingRegressor',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_bagging',
 '_base',
 '_forest',
 '_gb',
 '_gb_losses',
 '_gradient_boosting',
 '_hist_gradient_boosting',
 '_iforest',
 '_stacking',
 '_voting',
 '_weight_boosting']

In [ ]:
path+"\\mtcars.csv"




## 시험환경

In [6]:
# 출력을 원하실 경우 print() 함수 활용
# 예시) print(df.head())

# getcwd(), chdir() 등 작업 폴더 설정 불필요
# 파일 경로 상 내부 드라이브 경로(C: 등) 접근 불가

# 데이터 파일 읽기 예제
import pandas as pd
import numpy as np
X_test = pd.read_csv(path+"\\X_test.csv",encoding="euc-kr")

X_train = pd.read_csv(path+"\\X_train.csv",encoding="euc-kr")
y_train = pd.read_csv(path+"\\y_train.csv",encoding="euc-kr")


tr_len = len(X_train)
X_all = pd.concat([X_test,X_train] , axis= 0)


# 환불 여부만 파악
# X_all["환불여부"] = None # @ 비교해보기
# X_all.loc[X_all["환불금액"] != None,"환불여부"] = True

X_all = X_all.drop(["환불금액"],axis= 1) 
# print(X_all.head(5))
# print(X_all.info())

# print(X_all[["주구매상품","주구매지점"]].value_counts())
X_all = X_all.drop(["cust_id"],axis= 1) 
# X_all = X_all.drop(["주구매지점"],axis= 1) # @ 비교해보기


X_all_1 =  pd.get_dummies(X_all)

col = X_all_1.columns
print(col)
from sklearn.preprocessing import StandardScaler
# print(dir(sklearn.preprocessing))
import sklearn.preprocessing
# print(dir(sklearn.preprocessing))
Scaler = StandardScaler()
X_all_1 = Scaler.fit_transform(X_all_1)
# print(X_all_1.shape)
# print(col)
X_all_2 = pd.DataFrame(X_all_1,columns = col)

x_train = X_all_2[:tr_len]
x_test = X_all_2[tr_len:]

# @ 검증셋 만들기
a = np.floor(len(x_train)/10)
b= int(len(x_train)-a)
x_train_1 = x_train[:b]
x_val_1 = x_train[b:]
y_train_1 = y_train[:b]
y_val_1 = y_train[b:]


from sklearn.ensemble import RandomForestClassifier # GradientBoostingClassifier # RandomForestClassifier
from sklearn.svm import SVC # GradientBoostingClassifier # RandomForestClassifier
from xgboost import XGBClassifier
model = XGBClassifier(objective='binary:logistic' ,eval_metric='logloss')
# print(help(XGBClassifier))
model.fit(x_train_1,y_train_1["gender"],eval_set = [(x_val_1,y_val_1["gender"])])


pred= model.predict(x_val_1)

import sklearn.metrics
print(sklearn.metrics.roc_auc_score(y_val_1["gender"],pred),sklearn.metrics.f1_score(y_val_1["gender"],pred))


from xgboost import XGBClassifier
model = XGBClassifier(objective='binary:logistic' ,eval_metric='logloss')
# print(help(XGBClassifier))
model.fit(x_train,y_train["gender"])
print(model.score(x_train,y_train["gender"]))
prob= model.predict_proba(x_test)

df = pd.DataFrame({"custid":X_test.cust_id,"gender":prob[:,1]}) # @@ 
df.to_csv("003000000.csv",index = False)
# df = pd.read_csv("003000000.csv",index = False)
# print(df)
# # 답안 제출 참고
# 아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
# pd.DataFrame({'cust_id': X_test.cust_id, 'gender': pred}).to_csv('003000000.csv', index=False)

pred= model.predict(x_val_1)
import sklearn.metrics
print(sklearn.metrics.roc_auc_score(y_val_1["gender"],pred),sklearn.metrics.f1_score(y_val_1["gender"],pred))

Index(['총구매액', '최대구매액', '내점일수', '내점당구매건수', '주말방문비율', '구매주기', '주구매상품_가공식품',
       '주구매상품_가구', '주구매상품_건강식품', '주구매상품_골프', '주구매상품_구두', '주구매상품_기타',
       '주구매상품_남성 캐주얼', '주구매상품_남성 트랜디', '주구매상품_남성정장', '주구매상품_농산물', '주구매상품_대형가전',
       '주구매상품_디자이너', '주구매상품_란제리/내의', '주구매상품_명품', '주구매상품_모피/피혁', '주구매상품_보석',
       '주구매상품_생활잡화', '주구매상품_섬유잡화', '주구매상품_셔츠', '주구매상품_소형가전', '주구매상품_수산품',
       '주구매상품_스포츠', '주구매상품_시티웨어', '주구매상품_식기', '주구매상품_아동', '주구매상품_악기',
       '주구매상품_액세서리', '주구매상품_육류', '주구매상품_일용잡화', '주구매상품_젓갈/반찬', '주구매상품_주류',
       '주구매상품_주방가전', '주구매상품_주방용품', '주구매상품_차/커피', '주구매상품_축산가공', '주구매상품_침구/수예',
       '주구매상품_캐주얼', '주구매상품_커리어', '주구매상품_통신/컴퓨터', '주구매상품_트래디셔널', '주구매상품_피혁잡화',
       '주구매상품_화장품', '주구매지점_강남점', '주구매지점_관악점', '주구매지점_광주점', '주구매지점_노원점',
       '주구매지점_대구점', '주구매지점_대전점', '주구매지점_동래점', '주구매지점_미아점', '주구매지점_본  점',
       '주구매지점_부산본점', '주구매지점_부평점', '주구매지점_분당점', '주구매지점_상인점', '주구매지점_센텀시티점',
       '주구매지점_안양점', '주구매지점_영등포점', '주구매지점_울산점', '주구매지점_인천점', '주구매지점_일산점',
       '주구매지점_잠실점', '주구매지점_전주점', 

c:\users\happy\appdata\local\programs\python\python37\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23]	validation_0-logloss:0.73430
[24]	validation_0-logloss:0.73754
[25]	validation_0-logloss:0.73622
[26]	validation_0-logloss:0.73604
[27]	validation_0-logloss:0.73687
[28]	validation_0-logloss:0.74088
[29]	validation_0-logloss:0.74509
[30]	validation_0-logloss:0.74589
[31]	validation_0-logloss:0.75012
[32]	validation_0-logloss:0.75120
[33]	validation_0-logloss:0.75682
[34]	validation_0-logloss:0.75956
[35]	validation_0-logloss:0.75997
[36]	validation_0-logloss:0.76197
[37]	validation_0-logloss:0.76410
[38]	validation_0-logloss:0.76550
[39]	validation_0-logloss:0.76965
[40]	validation_0-logloss:0.76990
[41]	validation_0-logloss:0.77031
[42]	validation_0-logloss:0.77140
[43]	validation_0-logloss:0.77282
[44]	validation_0-logloss:0.77345
[45]	validation_0-logloss:0.77511
[46]	validation_0-logloss:0.77609
[47]	validation_0-logloss:0.77601
[48]	validation_0-logloss:0.77851
[49]	validation_0-logloss:0.78281
[50]	validation_0-logloss:0.78302
[51]	validation_0-logloss:0.78227
[52]	validatio

c:\users\happy\appdata\local\programs\python\python37\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.9428571428571428
0.9242933251891818 0.9166666666666667


In [5]:
## xgboost 검증셋 없이 학습

# 출력을 원하실 경우 print() 함수 활용
# 예시) print(df.head())

# getcwd(), chdir() 등 작업 폴더 설정 불필요
# 파일 경로 상 내부 드라이브 경로(C: 등) 접근 불가

# 데이터 파일 읽기 예제
import pandas as pd
X_test = pd.read_csv(path+"\\X_test.csv",encoding="euc-kr")

X_train = pd.read_csv(path+"\\X_train.csv",encoding="euc-kr")
y_train = pd.read_csv(path+"\\y_train.csv",encoding="euc-kr")

# 사용자 코딩

# 답안 제출 참고
# 아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
# pd.DataFrame({'cust_id': X_test.cust_id, 'gender': pred}).to_csv('003000000.csv', index=False)

import numpy as np
X_all = pd.concat([X_train,X_test],axis = 0)
X_train_len = len(X_train)

# print(len(X_train))
# print(len(X_test))
# print(len(X_all))

train_all = pd.concat([X_train,y_train],axis = 1)
# print(len(train_all))
# print(X_all)


#상관분석 후 사용할 컬럼 지정
aa = pd.DataFrame(abs(train_all.corr()['gender'])>0.02)
use_col = list(aa.loc[aa['gender']==True][:-1].index)
print(use_col)
######

#더미화 및 데이터 확인
X_all_1 = X_all[use_col]
# print(X_all_1.info())
X_all_2 = pd.get_dummies(X_all_1)
# print(X_all_2.info())

#정규화
from sklearn.preprocessing import StandardScaler
SS = StandardScaler()
X_all_2 = SS.fit_transform(X_all_2)
# print(X_all_2)


#데이터 프레임 다시 나누기
# print(X_train_len)

X_train_2 = X_all_2[:X_train_len]
X_test_2 = X_all_2[X_train_len:]
y_train_2 = y_train['gender']
# print(y_train_2)

from xgboost import XGBClassifier
# print(dir(XGBClassifier))
# model = XGBClassifier(eval_metric = "logloss",use_label_encoder = False)
model = XGBClassifier(eval_metric = "logloss",use_label_encoder = False)
# print(help(model))
model.fit(X_train_2,y_train_2)

print(model.score(X_train_2,y_train['gender'])) # 모델 점수

pred = model.predict(X_test_2)
# prob = model.predict_proba(X_test_2)[:,:1]
pd.DataFrame({'cust_id': X_test.cust_id, 'gender': pred}).to_csv('003000000.csv', index=False)


['총구매액', '최대구매액', '환불금액', '내점일수', '내점당구매건수', '주말방문비율', '구매주기']
0.932


In [2]:
######################################################################### 
## xgboost 검증셋 추가

# 출력을 원하실 경우 print() 함수 활용
# 예시) print(df.head())

# getcwd(), chdir() 등 작업 폴더 설정 불필요
# 파일 경로 상 내부 드라이브 경로(C: 등) 접근 불가

# 데이터 파일 읽기 예제
import pandas as pd
X_test = pd.read_csv(path+"\\X_test.csv",encoding="euc-kr")

X_train = pd.read_csv(path+"\\X_train.csv",encoding="euc-kr")
y_train = pd.read_csv(path+"\\y_train.csv",encoding="euc-kr")

# 사용자 코딩

# 답안 제출 참고
# 아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
# pd.DataFrame({'cust_id': X_test.cust_id, 'gender': pred}).to_csv('003000000.csv', index=False)

import numpy as np
X_all = pd.concat([X_train,X_test],axis = 0)
X_train_len = len(X_train)

# print(len(X_train))
# print(len(X_test))
# print(len(X_all))

train_all = pd.concat([X_train,y_train],axis = 1)
# print(len(train_all))
# print(X_all)


#상관분석 후 사용할 컬럼 지정
aa = pd.DataFrame(abs(train_all.corr()['gender'])>0.02)
use_col = list(aa.loc[aa['gender']==True][:-1].index)
print(use_col)
######

#더미화 및 데이터 확인
X_all_1 = X_all[use_col]
# print(X_all_1.info())
X_all_2 = pd.get_dummies(X_all_1)
# print(X_all_2.info())

#정규화
from sklearn.preprocessing import StandardScaler
SS = StandardScaler()
X_all_2 = SS.fit_transform(X_all_2)
# print(X_all_2)


#데이터 프레임 다시 나누기
# print(X_train_len)
X_test_2 = X_all_2[X_train_len:]
X_train_2 = X_all_2[:X_train_len]
y_train_2 = y_train['gender']

x_validation = X_train_2[-500:]
y_validation = y_train_2[-500:]
print(len(x_validation))

X_train_2 = X_train_2[:-500]
y_train_2 = y_train_2[:-500]

# print(y_train_2)

from xgboost import XGBClassifier
# print(dir(XGBClassifier))
# model = XGBClassifier(eval_metric = "logloss",use_label_encoder = False)
model = XGBClassifier(eval_metric = "logloss",use_label_encoder = False)
# print(help(model))
# model.fit(X_train_2,y_train_2)
model.fit(X_train_2,y_train_2,eval_set=[(x_validation, y_validation)])

print(model.score(X_train_2,y_train_2)) # 모델 점수

pred = model.predict(X_test_2)
# prob = model.predict_proba(X_test_2)[:,:1]
pd.DataFrame({'cust_id': X_test.cust_id, 'gender': pred}).to_csv('003000000.csv', index=False)




pred= model.predict(x_validation)

import sklearn.metrics
print(sklearn.metrics.roc_auc_score(y_validation,pred),sklearn.metrics.f1_score(y_validation,pred))

['총구매액', '최대구매액', '환불금액', '내점일수', '내점당구매건수', '주말방문비율', '구매주기']
500
[0]	validation_0-logloss:0.67263
[1]	validation_0-logloss:0.66471
[2]	validation_0-logloss:0.66166
[3]	validation_0-logloss:0.65971
[4]	validation_0-logloss:0.65713
[5]	validation_0-logloss:0.65694
[6]	validation_0-logloss:0.65899
[7]	validation_0-logloss:0.65547
[8]	validation_0-logloss:0.65613
[9]	validation_0-logloss:0.66075
[10]	validation_0-logloss:0.66352
[11]	validation_0-logloss:0.66451
[12]	validation_0-logloss:0.66231
[13]	validation_0-logloss:0.66534
[14]	validation_0-logloss:0.66675
[15]	validation_0-logloss:0.66750
[16]	validation_0-logloss:0.66787
[17]	validation_0-logloss:0.66857
[18]	validation_0-logloss:0.66907
[19]	validation_0-logloss:0.67068
[20]	validation_0-logloss:0.67277
[21]	validation_0-logloss:0.67333
[22]	validation_0-logloss:0.67401
[23]	validation_0-logloss:0.67356
[24]	validation_0-logloss:0.67435
[25]	validation_0-logloss:0.67556
[26]	validation_0-logloss:0.67727
[27]	validation_0-logloss

In [ ]:
######################################################################### 
## xgboost 검증셋 추가

# 출력을 원하실 경우 print() 함수 활용
# 예시) print(df.head())

# getcwd(), chdir() 등 작업 폴더 설정 불필요
# 파일 경로 상 내부 드라이브 경로(C: 등) 접근 불가

# 데이터 파일 읽기 예제
import pandas as pd
X_test = pd.read_csv(path+"\\X_test.csv",encoding="euc-kr")

X_train = pd.read_csv(path+"\\X_train.csv",encoding="euc-kr")
y_train = pd.read_csv(path+"\\y_train.csv",encoding="euc-kr")
# 사용자 코딩

# 답안 제출 참고
# 아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
# pd.DataFrame({'cust_id': X_test.cust_id, 'gender': pred}).to_csv('003000000.csv', index=False)

import numpy as np
X_all = pd.concat([X_train,X_test],axis = 0)
X_train_len = len(X_train)

# print(len(X_train))
# print(len(X_test))
# print(len(X_all))

train_all = pd.concat([X_train,y_train],axis = 1)
# print(len(train_all))
# print(X_all)


#상관분석 후 사용할 컬럼 지정
aa = pd.DataFrame(abs(train_all.corr()['gender'])>0.02)
use_col = list(aa.loc[aa['gender']==True][:-1].index)
print(use_col)
######

#더미화 및 데이터 확인
X_all_1 = X_all[use_col]
X_all_1 = X_all_1.drop(["환불금액"], axis = 1)
print(X_all_1.info())
X_all_2 = pd.get_dummies(X_all_1)
# print(X_all_2.info())

#정규화
from sklearn.preprocessing import StandardScaler
SS = StandardScaler()
X_all_2 = SS.fit_transform(X_all_2)
# print(X_all_2)


#데이터 프레임 다시 나누기
# print(X_train_len)
X_test_2 = X_all_2[X_train_len:]
X_train_2 = X_all_2[:X_train_len]
y_train_2 = y_train

x_validation = X_train_2[-500:]
y_validation = y_train_2[-500:]
print(len(x_validation))

X_train_2 = X_train_2[:-500]
y_train_2 = y_train_2[:-500]

from sklearn.ensemble import RandomForestClassifier # GradientBoostingClassifier # RandomForestClassifier
# print(dir(XGBClassifier))
# model = XGBClassifier(eval_metric = "logloss",use_label_encoder = False)
model = RandomForestClassifier()
model.fit(X_train_2,y_train_2["gender"])

pred= model.predict(x_validation)

import sklearn.metrics
print(sklearn.metrics.roc_auc_score(y_validation["gender"],pred),sklearn.metrics.f1_score(y_validation["gender"],pred))